## **Notebook 1 — ETL (Extract, Transform, Load)**

**1. Load Libraries & Setup**

In [10]:
# 1) IMPORTS & PATHS
import os
import re
import ast
import pandas as pd
import numpy as np
from dateutil import parser

# Show where this notebook is running from (helps debug path issues)
print("Current working directory:", os.getcwd())

# Because the notebook is in jupyter_notebooks/, go up one level to reach data/
RAW_PATH = "../data/raw/dataset.csv"
CLEAN_PATH = "../data/processed/spotify_clean.csv"
DATA_DICTIONARY_PATH = "../data/processed/data_dictionary.csv"



Current working directory: /Users/maria_marsella/Downloads/spotify-streaming-analysis-main/jupyter_notebooks


**2. Extract: Load Raw Data**

In [11]:
# 2) LOAD RAW CSV & PREVIEW
df = pd.read_csv(RAW_PATH)

# Basic shape and first few rows
print("Rows x Columns:", df.shape)
display(df.head())

# Show exact column names present (useful for renaming later)
print("Columns:", df.columns.tolist())



Rows x Columns: (114000, 21)


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


Columns: ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']


**3. Standardise Column Names**

In [12]:
# 3) STANDARDIZE / RENAME COLUMNS
# Map any alternative names you see in your df.columns to the canonical names we will use.
rename_map = {
    # left = current name you see, right = target name we want
    "released": "release_date",
    "release": "release_date",
    "date": "release_date",
    "artist": "artists",        # in case you have a single 'artist' column
    "duration_ms.": "duration_ms",
    # add more if your dataset uses different headers
}

df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})
print("Columns after standardization:", df.columns.tolist())


Columns after standardization: ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre']


**4. Inspect Missing Values & Data Types**

In [13]:
# 4) INSPECT MISSING VALUES & DTYPES
df.info()  # prints dtypes and non-null counts

# Percent missing per column (sorted high → low)
missing_pct = (df.isna().mean().sort_values(ascending=False) * 100).round(2)
display(missing_pct.to_frame("missing_%"))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

,missing_%
artists,0.0
album_name,0.0
track_name,0.0
Unnamed: 0,0.0
mode,0.0
time_signature,0.0
tempo,0.0
valence,0.0
liveness,0.0
instrumentalness,0.0


**5. Artists parsing & Safe date**

- parse_artists converts a string like "['Drake','Future']" or "Drake, Future" into a clean Python list.

- safe_date robustly parses dates; invalid ones become NaT.

In [17]:
ARTIST_SPLIT_RE = re.compile(r";|,|\band\b|&")

def parse_artists(artists_raw):
    """Convert artist text into a list of clean artist names."""
    if not isinstance(artists_raw, str) or not artists_raw.strip():
        return []
    # Try list literal first (e.g., "['Drake','Future']")
    try:
        val = ast.literal_eval(artists_raw)
        if isinstance(val, list):
            return [str(x).strip() for x in val if str(x).strip()]
    except Exception:
        pass
    # Fallback: split on common delimiters
    return [p.strip() for p in ARTIST_SPLIT_RE.split(artists_raw) if p.strip()]

def safe_date(x):
    """Parse a date safely; return NaT if it fails."""
    try:
        return parser.parse(str(x))
    except Exception:
        return pd.NaT


**6. Parse artists → add artists_list, primary_artist, n_artists**

In [15]:
if "artists" in df.columns:
    df["artists_list"] = df["artists"].apply(parse_artists)
    df["primary_artist"] = df["artists_list"].apply(lambda xs: xs[0] if xs else None)
    df["n_artists"] = df["artists_list"].apply(len)
else:
    # Create empty columns if 'artists' is missing
    df["artists_list"] = [[] for _ in range(len(df))]
    df["primary_artist"] = None
    df["n_artists"] = 0

display(df[["artists"]].head() if "artists" in df.columns else df.head())
display(df[["primary_artist","n_artists"]].head())


<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal


,artists
0,Gen Hoshino
1,Ben Woodward
2,Ingrid Michaelson;ZAYN
3,Kina Grannis
4,Chord Overstreet


,primary_artist,n_artists
0,Gen Hoshino,1
1,Ben Woodward,1
2,Ingrid Michaelson,2
3,Kina Grannis,1
4,Chord Overstreet,1


**7. Parse release_date → create year & month**

We convert release_date to datetime and derives year/month. If the dataset doesn’t have release_date, this cell will just print the available columns so we can adjust the rename map above.

In [23]:
from dateutil import parser

def safe_date(x):
    """Convert string to datetime if possible, else return NaT (missing date)."""
    try:
        return parser.parse(str(x))
    except:
        return pd.NaT

# --- Check if column exists ---
if "release_date" in df.columns:
    # Parse release_date and extract year/month
    df["release_date"] = df["release_date"].apply(safe_date)
    df["year"] = df["release_date"].dt.year
    df["month"] = df["release_date"].dt.month
    print("✅ Parsed release_date successfully")

    # Show preview (release_date available)
    display(df[["release_date", "year", "month"]].head())

else:
    # If release_date is missing, just confirm and skip
    print("⚠️ 'release_date' column not found. Available columns:")
    print(df.columns.tolist())

    # Ensure year/month columns exist so later code doesn't fail
    if "year" not in df.columns:
        df["year"] = pd.NA
    if "month" not in df.columns:
        df["month"] = pd.NA

    # Display only the columns that exist
    display(df[["year", "month"]].head())




⚠️ 'release_date' column not found. Available columns:
['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre', 'artists_list', 'primary_artist', 'n_artists', 'year', 'month']


,year,month
0,<NA>,<NA>
1,<NA>,<NA>
2,<NA>,<NA>
3,<NA>,<NA>
4,<NA>,<NA>


** 8.Type coercion for numeric/boolean columns**

We ensure numeric columns are numbers (not strings) and convert explicit to boolean. Errors become NaN so they don’t break plots.

In [24]:
numeric_cols = [
    "popularity","duration_ms","danceability","energy","key","loudness","mode",
    "speechiness","acousticness","instrumentalness","liveness","valence","tempo",
    "time_signature"
]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# explicit → boolean (handles 'True'/'False'/'1'/'0')
if "explicit" in df.columns:
    tmp = df["explicit"].astype(str).str.lower().map(
        {"true": True, "false": False, "1": True, "0": False}
    )
    # if mapping created many NaNs (weird encoding), try numeric then bool
    if tmp.notna().mean() < 0.2:
        df["explicit"] = pd.to_numeric(df["explicit"], errors="coerce").fillna(0).astype(int).astype(bool)
    else:
        df["explicit"] = tmp


**9.Handle missing values (simple, explainable rules)**

- Fill popularity with mean (so visuals don’t drop rows).
- Drop rows with missing name (critical for identification).
- Keep other NaNs (analyst can decide later).

In [25]:
# 9) HANDLE MISSING VALUES (MINIMAL, EXPLAINABLE)
if "popularity" in df.columns:
    df["popularity"] = df["popularity"].fillna(df["popularity"].mean())

# Drop rows missing the track name (core identifier)
if "name" in df.columns:
    df = df.dropna(subset=["name"])


**10. Deduplicate rows**

Removes duplicates by id when available, otherwise by (name, primary_artist).

In [26]:
before = len(df)

if "id" in df.columns:
    # Keep the most popular version if duplicates share the same id
    df = df.sort_values("popularity", ascending=False).drop_duplicates("id")

if {"name","primary_artist"}.issubset(df.columns):
    df = df.drop_duplicates(["name","primary_artist"])

after = len(df)
print(f"Deduplicated: {before:,} → {after:,} rows")


Deduplicated: 114,000 → 114,000 rows


**8. Transform: Convert Explicit Column**

In [27]:
if "explicit" in df.columns:
    df["explicit"] = (
        df["explicit"]
        .astype(str)
        .str.lower()
        .map({"true": True, "1": True, "false": False, "0": False})
    )


**1. Validate Data Quality**

In [28]:
issues = []

if "popularity" in df.columns:
    bad = ~df["popularity"].between(0, 100)
    if bad.any():
        issues.append(f"Popularity out of [0,100]: {bad.sum()} rows")

if "duration_ms" in df.columns:
    bad = df["duration_ms"].notna() & (df["duration_ms"] <= 0)
    if bad.any():
        issues.append(f"Non-positive duration_ms: {bad.sum()} rows")

if "tempo" in df.columns:
    # tempo outside 40–220 BPM is unusual (not always wrong)
    out = df["tempo"].notna() & ~df["tempo"].between(40, 220)
    if out.mean() > 0.10:
        issues.append(f"Tempo outliers >10% of data: {out.sum()} rows")

print("Validation issues:" if issues else "Validation passed ✅")
for m in issues:
    print(" -", m)


Validation issues:
 - Non-positive duration_ms: 1 rows


**12. Load: Export Clean Dataset**

In [29]:
# Ensure the output folder exists
os.makedirs(os.path.dirname(CLEAN_PATH), exist_ok=True)

df.to_csv(CLEAN_PATH, index=False)
print(f"Clean dataset saved → {CLEAN_PATH}  (rows: {len(df):,})")



Clean dataset saved → ../data/processed/spotify_clean.csv  (rows: 114,000)
